In [8]:
from __future__ import print_function

import os
import base64
import pandas as pd
import json as json
import numpy as np

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

from google.oauth2 import service_account

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

def calculate_calorie_percent(x,y):
    return (x-y)/x*100

def calculate_calorie_diff(x,y):
    return (x-y)

def make_data_frame_from_list(list):
    cols = list.pop(0)
    df = pd.DataFrame(data=list, columns=cols)
    dfCols = df.columns

    # Set the Date column to be date time dType
    df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)

    string_cols = ['Programme', 'Phase', 'Healthy or Sick', 'Book', 'Fast Type']
    ignore_cols = ['Date'] + string_cols

    ## Go through columns we want to convert to numbers
    cols_to_num = [i for i in df.columns if i not in ignore_cols]
    for col in cols_to_num:
        df[col] = df[col].apply(pd.to_numeric, errors='coerce')

    ## Go through columns we want to conver to strings
    for col in string_cols:
        df[col] = df[col].astype(str)

    # Set the index of the data frame to be a date
    df.set_index(['Date'], inplace=True)

    df['Calories Percent'] = df.apply(lambda x: calculate_calorie_percent(x['Calories Consumed'], x['Calories Burned']), axis=1)
    df['Maintenance Calorie Diff'] = df.apply(lambda x: calculate_calorie_diff(x['Calories Consumed'], x['Calories Burned']), axis=1)

    return df

# If modifying these scopes, delete the file token.json.

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

SERVICE_ACCOUNT_FILE = 'keys-local.json'

credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)

SAMPLE_SPREADSHEET_ID = '1ohNM7O8Ecg1EtI3WLv3MbyuKLP7gm7WQFufcqylxeUQ'
# SAMPLE_RANGE_NAME = 'Daily Log Test!A1:N498'
SAMPLE_RANGE_NAME = 'Daily Log!A1:AA600'

service = build('sheets', 'v4', credentials=credentials)

# Call the Sheets API
daily_log_sheet = service.spreadsheets()
daily_log_results = daily_log_sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME).execute()
daily_log_values = daily_log_results.get('values', [])

if not daily_log_values:
    print('No data found.')
    exit()

total_number_columns = len(daily_log_values[0])

validated_list_for_data_frame = []
validated_list_for_data_frame.append(daily_log_values[0])

for i in range(1, len(daily_log_values)):
    # Fill the rest of the list with empty values
    number_columns = len(daily_log_values[i])
    daily_log_values[i].extend([''] * (total_number_columns - number_columns))
    validated_list_for_data_frame.append(daily_log_values[i])

df = make_data_frame_from_list(validated_list_for_data_frame)

def single_line_graph(df, column, weeks):
    now = datetime.now().date()

    monday_this_week = now - timedelta(days = now.weekday())
    sunday_this_week = monday_this_week + timedelta(days=6)

    monday_12_weeks_ago = monday_this_week - timedelta(weeks=weeks)

    df2 = df.query("Date >= @monday_12_weeks_ago and Date <= @now") \
    .groupby(pd.Grouper(freq='W', level='Date'))[column].mean(numeric_only=True)

    reformatted_data = []
    for index, item in df2.items():
            points = {}
            points["x"] = index.strftime("%d-%b")
            points["y"] = np.nan_to_num(item)
            reformatted_data.append(points)

    return reformatted_data

def single_line_graph_from_date(df, column, weeks, from_date):

    monday_from_date = from_date - timedelta(days = from_date.weekday())
    sunday_from_date = monday_from_date + timedelta(days=6)

    monday_in_n_weeks = monday_from_date + timedelta(weeks=weeks)

    df2 = df.query("Date < @monday_in_n_weeks and Date >= @monday_from_date") \
    .groupby(pd.Grouper(freq='W', level='Date'))[column].mean(numeric_only=True)

    reformatted_data = []
    for index, item in df2.items():
            points = {}
            points["x"] = index.strftime("%d-%b")
            points["y"] = np.nan_to_num(item)
            reformatted_data.append(points)

    return reformatted_data


def heatmap_data(df, column, months, value_as_num=True, show_historical_data=True, oldest_first=True):
    output_data = []

    for n in range(0,months):
        if (show_historical_data):
            start_month = (datetime.now().date() - relativedelta(months=n)).replace(day=1)
        else: 
            start_month = (datetime.now().date() + relativedelta(months=n)).replace(day=1)

        end_month = start_month + relativedelta(months=1)

        data = df.query("Date >= @start_month and Date < @end_month")[column]

        month_data = {}
        month_data["name"] = start_month.strftime("%B")

        # pad out the array with 0's if the data starts mid month
        reformatted_data = []
        day_indexes = data.head(1).index.day.tolist()
        if (len(day_indexes) > 0):
            first_day = data.head(1).index.day.tolist()[0]

            if first_day > 1:
                for i in range(1, first_day):
                    points = {}
                    points["x"] = str(i)
                    points["y"] = 0
                    reformatted_data.append(points)

        for index, item in data.items():
            points = {}
            points["x"] = index.strftime("%d")
            points["y"] = np.nan_to_num(item) if value_as_num == True else item
            reformatted_data.append(points)

        month_data["data"] = reformatted_data

        output_data.append(month_data)

    if (oldest_first == True):
        return output_data # Returns the list with oldest to newest
    else:
        return output_data[::-1] # Returns the list newest to oldest

In [18]:
def most_word_occurrence(df, column, weeks, from_date):

    monday_from_date = from_date - timedelta(days = from_date.weekday())
    sunday_from_date = monday_from_date + timedelta(days=6)

    monday_in_n_weeks = monday_from_date + timedelta(weeks=weeks)

    df2 = df.query("Date < @monday_in_n_weeks and Date >= @monday_from_date") \
    .groupby(pd.Grouper(freq='W', level='Date'))[column].apply(lambda x: x.value_counts().index[0])

    reformatted_data = []
    for index, item in df2.items():
            points = {}
            points["x"] = index.strftime("%d-%b")
            points["y"] = np.nan_to_num(item)
            reformatted_data.append(points)

    return reformatted_data

health_sick = most_word_occurrence(df, 'Healthy or Sick', 18, datetime.strptime("01012023", "%d%m%Y"))
print(health_sick)




Date
2023-01-01           
2023-01-08    Healthy
2023-01-15    Healthy
2023-01-22    Healthy
2023-01-29    Healthy
2023-02-05    Healthy
2023-02-12     Travel
2023-02-19       Sick
2023-02-26       Sick
2023-03-05       Sick
2023-03-12           
2023-03-19           
2023-03-26           
2023-04-02           
2023-04-09           
2023-04-16           
2023-04-23           
2023-04-30           
Freq: W-SUN, Name: Healthy or Sick, dtype: object


In [ ]:
health_sick = average_word_occurrence(df, 'Healthy Or Sick', 18, datetime.strptime("01012023", "%d%m%Y"))

health_sick

# json_string = json.dumps(out)

# with open('../../static/chart-data/data.json','w') as outfile:
#     outfile.write(json_string)